In [ ]:
##### Audio Classification Using Machine Learning
#####dataset url https://urbansounddataset.weebly.com/urbansound8k.html

In [ ]:
!pip install librosa

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
filename='../input/birdclef-2022/train_audio/akiapo/XC122401.ogg'

In [ ]:
import IPython.display as ipd
import librosa
import librosa.display

In [ ]:
!dir

In [ ]:
### Dog Sound
plt.figure(figsize=(14,5))
data,sample_rate=librosa.load(filename)
librosa.display.waveshow(data,sr=sample_rate)
ipd.Audio(filename)

In [ ]:
### Sound
filename='../input/birdclef-2022/train_audio/akekee/XC174953.ogg'
plt.figure(figsize=(14,5))
data,sample_rate=librosa.load(filename)
librosa.display.waveshow(data,sr=sample_rate)
ipd.Audio(filename)

In [ ]:
sample_rate


In [ ]:
# from scipy.io import wavfile as wav
# wave_sample_rate, wave_audio=wav.read(filename)


In [ ]:
# wave_sample_rate

In [ ]:
# wave_audio

In [ ]:
data

In [ ]:
import pandas as pd

metadata1=pd.read_csv('../input/birdclef-2022/eBird_Taxonomy_v2021.csv')
metadata1.head(10)

In [ ]:
metadata1.isnull().sum()
metadata1['REPORT_AS'].isnull().round()*100

In [ ]:
### Check whether the dataset is imbalanced
metadata1['REPORT_AS'].value_counts()

In [ ]:
### Let's read a sample audio using librosa
import librosa
audio_file_path='../input/birdclef-2022/train_audio/afrsil1/XC125458.ogg'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)
print(mfccs.shape)

In [ ]:
mfccs

In [ ]:
#### Extracting MFCC's For every audio file
import pandas as pd
import os
import librosa

audio_dataset_path='../input/birdclef-2022/train_audio/'
metadata=pd.read_csv('../input/birdclef-2022/train_metadata.csv')
metadata.head()

In [ ]:
metadata.isnull().sum()

In [ ]:
master = pd.merge(metadata,metadata1,how="inner",left_on="primary_label",right_on="REPORT_AS")
master.head(5)

In [ ]:
master.shape

In [ ]:
master.columns

In [ ]:
master.isnull().sum()

# 

In [ ]:
master.drop(['SPECIES_GROUP','REPORT_AS'],axis=1)

In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [ ]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path)+'/',str(row["filename"]))
    final_class_labels=row["primary_label"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','primary_label'])
extracted_features_df.head()

In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['primary_label'].tolist())

In [ ]:
X.shape

In [ ]:
y

In [ ]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y

In [ ]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


In [ ]:
X_train

In [ ]:
y

In [ ]:
X_train.shape


In [ ]:
X_test.shape


In [ ]:
y_train.shape


In [ ]:
y_test.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
#Measure the performance of the decision tree model
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_test,predictions))
np.argmax(y_test, axis=1)
# print(confusion_matrix(y_test,predictions))
confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

In [ ]:
score = model.score(X_test, y_test)
score

### Random Forest

In [ ]:
# Random forest
# Fit
# Import model object
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score, precision_score, recall_score

# Create model object
rfc = RandomForestClassifier(n_estimators = 200)

# Fit model to training data
rfc.fit(X_train,y_train)
# Predict
y_pred_rfc = rfc.predict(X_test)

# Classification report
print('--'*40)
print('Classification Report')
# print(classification_report(y_test, y_pred_rfc))
confusion_matrix(y_test.argmax(axis=1), y_pred_rfc.argmax(axis=1))
# Accuracy
print('--'*40)
rf_accuracy = round(accuracy_score(y_test, y_pred_rfc)*100,2)
print('Accuracy', rf_accuracy,'%')

In [ ]:
# Score It
print('Random Forest')
# Confusion matrix
print('\n')
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred_rfc))

# Classification report
print('--'*40)
print('Classification Report')
# print(classification_report(y_test, y_pred_rfc))
print(confusion_matrix(y_test.argmax(axis=1), y_pred_rfc.argmax(axis=1)))
y_
# Accuracy
print('--'*40)
rf_accuracy = round(accuracy_score(y_test, y_pred_rfc)*100,2)
print('Accuracy', rf_accuracy,'%')
# Predict
y_pred_rfc = rfc.predict(X_test)
# Score It
print('Random Forest')
# Confusion matrix
print('\n')
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred_rfc))


In [ ]:
# Fit new KNN
# Create model object
knn = KNeighborsClassifier(n_neighbors = 13)

# Fit new KNN on training data
knn.fit(X_train, y_train)

In [ ]:
# Predict new KNN
y_pred_knn_op = knn.predict(X_test)
# Score it with new KNN
print('K-Nearest Neighbors(KNN)')
print('k = 13')

# Confusion Matrix
print('\n')
print(confusion_matrix(y_test, y_pred_knn_op))

# Classification Report
print('--'*40)
print('Classfication Report',classification_report(y_test, y_pred_knn_op))

# Accuracy
print('--'*40)
knn_op_accuracy =round(accuracy_score(y_test, y_pred_knn_op)*100,2)
print('Accuracy',knn_op_accuracy,'%')


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics


In [ ]:
### No of classes
num_labels=y.shape[1]

In [ ]:
model=Sequential()
###first layer
model.add(Dense(1000,input_shape=(40,)))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))



In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 10
num_batch_size = 8

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
prediction_feature.shape

In [ ]:
X_test[1]

In [ ]:
model.predict_classes(X_test)

In [ ]:
filename="../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class